# **Cancer Histology Image Classification UsingResNet50 - Deep Learning Model Overview**  
***
ResNet50 (Residual Network with 50 layers) is a deep convolutional neural network (CNN) widely used for **image classification, object detection, and feature extraction**. It was introduced by Microsoft researchers in 2015 in the paper **"Deep Residual Learning for Image Recognition"**, significantly improving deep network training by addressing the **vanishing gradient problem**.  

## **Key Features of ResNet50:**  
**Residual Learning:** Uses **skip connections** to bypass layers, improving gradient flow and training stability.  
**Deep Architecture:** Consists of **50 layers**, allowing for better feature extraction than shallow networks.  
**Pre-trained Weights:** Can use **ImageNet weights**, enhancing transfer learning capabilities.  
**Improved Training:** Residual connections mitigate **vanishing gradients**, making deep networks trainable.  

## **Why Use ResNet50 Instead of VGG16?**  
**Better Performance:** ResNet50 **outperforms VGG16** in accuracy while using fewer parameters (~25M vs. ~138M).  
**Deeper Yet Efficient:** ResNet50 achieves better results **without excessive computational cost**.  
**Faster Convergence:** Residual connections enable **better backpropagation**, allowing deeper architectures.  

## **ResNet50 in Our Project**  
**Replacing VGG16 with ResNet50** while maintaining the dataset split (60% training, 10% validation, 30% testing).  
Using **pre-trained ResNet50 as a feature extractor**, freezing convolutional layers, and adding a custom classifier.  
Performing **binary classification** (Benign vs. Malignant) on histopathology images.  
Evaluating performance using **Accuracy, F1-Score, G-Mean, and Informedness (IBA).**  

This implementation ensures a **more efficient and accurate classification** of histopathology images while leveraging deep residual learning. 🚀


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


***
**Magnification Factor: 40X**
***

In [ ]:
import os
import tensorflow as tf
import numpy as np
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from imblearn.metrics import geometric_mean_score

# Define dataset paths
benign_dir = "/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/benign/40X"
malignant_dir = "/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/malignant/40X"

# Function to load image paths
def load_image_paths(dir_path):
    return [os.path.join(dir_path, img) for img in os.listdir(dir_path) if img.endswith('.png')]

benign_images = load_image_paths(benign_dir)
malignant_images = load_image_paths(malignant_dir)

print(f"Total Benign Images: {len(benign_images)}")
print(f"Total Malignant Images: {len(malignant_images)}")

# Create labels (0 = Benign, 1 = Malignant)
benign_labels = [0] * len(benign_images)
malignant_labels = [1] * len(malignant_images)

# Combine images and labels
all_images = np.array(benign_images + malignant_images)
all_labels = np.array(benign_labels + malignant_labels)

# Split into training (70%) and testing (30%)
train_images, test_images, train_labels, test_labels = train_test_split(
    all_images, all_labels, test_size=0.3, stratify=all_labels, random_state=42
)

# Further split training set into 60% training and 10% validation
train_images, val_images, train_labels, val_labels = train_test_split(
    train_images, train_labels, test_size=0.1429, stratify=train_labels, random_state=42
)

print(f"Training samples: {len(train_images)}")
print(f"Validation samples: {len(val_images)}")
print(f"Testing samples: {len(test_images)}")

# Function to preprocess images
def process_path(file_path, label):
    img = tf.io.read_file(file_path)
    img = tf.image.decode_png(img, channels=3)  # Decode PNG images
    img = tf.image.resize(img, [224, 224])  # Resize to 224x224
    img = img / 255.0  # Normalize pixel values
    return img, label

# Create TensorFlow datasets
BATCH_SIZE = 128

train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
train_dataset = train_dataset.map(process_path).shuffle(1000).batch(BATCH_SIZE)

val_dataset = tf.data.Dataset.from_tensor_slices((val_images, val_labels))
val_dataset = val_dataset.map(process_path).batch(BATCH_SIZE)

test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels))
test_dataset = test_dataset.map(process_path).batch(BATCH_SIZE)

# Load ResNet50 without the top classification layer
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the pre-trained layers
for layer in base_model.layers:
    layer.trainable = False

# Add custom classifier on top
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(128, activation='relu')(x)
x = Dense(1, activation='sigmoid')(x)  # Binary classification

# Create the model
model = Model(inputs=base_model.input, outputs=x)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train model
EPOCHS = 100
history = model.fit(train_dataset, validation_data=val_dataset, epochs=EPOCHS)

# Evaluate model
import sklearn.metrics as metrics
from sklearn.metrics import confusion_matrix

# Predict on test data
test_preds = model.predict(test_dataset)
test_preds = (test_preds > 0.5).astype(int).flatten()

# Get confusion matrix values
tn, fp, fn, tp = confusion_matrix(test_labels, test_preds).ravel()

# Calculate IBA
iba = (tp / (tp + fn)) + (tn / (tn + fp)) - 1

# Output results
f1 = f1_score(test_labels, test_preds)
gmean = geometric_mean_score(test_labels, test_preds)

loss, accuracy = model.evaluate(test_dataset)
print(f"Test Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"G-Mean: {gmean:.4f}")
print(f"Informedness (IBA): {iba:.4f}")


Total Benign Images: 626
Total Malignant Images: 1370
Training samples: 1197
Validation samples: 200
Testing samples: 599
94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step
Epoch 1/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 232s 6s/step - accuracy: 0.5879 - loss: 0.6837 - val_accuracy: 0.6850 - val_loss: 0.6601
Epoch 2/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 18s 612ms/step - accuracy: 0.6917 - loss: 0.6639 - val_accuracy: 0.5900 - val_loss: 0.6733
Epoch 3/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 22s 667ms/step - accuracy: 0.6288 - loss: 0.6448 - val_accuracy: 0.6850 - val_loss: 0.6092
Epoch 4/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 17s 655ms/step - accuracy: 0.6783 - loss: 0.6272 - val_accuracy: 0.6850 - val_loss: 0.6100
Epoch 5/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 21s 590ms/step - accuracy: 0.6967 - loss: 0.5998 - val_accuracy: 0.6850 - val_loss: 0.6052
Epoch 6/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 21s 551ms/step - accuracy: 0.6823 - loss: 0.6118 - val_accuracy: 0.6850 - val_loss: 0.6050
Epoch 7/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 20s 675ms/

***
**Magnification Factor: 100X**
***

In [ ]:
import os
import tensorflow as tf
import numpy as np
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from imblearn.metrics import geometric_mean_score

# Define dataset paths
benign_dir = "/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/benign/100X"
malignant_dir = "/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/malignant/100X"

# Function to load image paths
def load_image_paths(dir_path):
    return [os.path.join(dir_path, img) for img in os.listdir(dir_path) if img.endswith('.png')]

benign_images = load_image_paths(benign_dir)
malignant_images = load_image_paths(malignant_dir)

print(f"Total Benign Images: {len(benign_images)}")
print(f"Total Malignant Images: {len(malignant_images)}")

# Create labels (0 = Benign, 1 = Malignant)
benign_labels = [0] * len(benign_images)
malignant_labels = [1] * len(malignant_images)

# Combine images and labels
all_images = np.array(benign_images + malignant_images)
all_labels = np.array(benign_labels + malignant_labels)

# Split into training (70%) and testing (30%)
train_images, test_images, train_labels, test_labels = train_test_split(
    all_images, all_labels, test_size=0.3, stratify=all_labels, random_state=42
)

# Further split training set into 60% training and 10% validation
train_images, val_images, train_labels, val_labels = train_test_split(
    train_images, train_labels, test_size=0.1429, stratify=train_labels, random_state=42
)

print(f"Training samples: {len(train_images)}")
print(f"Validation samples: {len(val_images)}")
print(f"Testing samples: {len(test_images)}")

# Function to preprocess images
def process_path(file_path, label):
    img = tf.io.read_file(file_path)
    img = tf.image.decode_png(img, channels=3)  # Decode PNG images
    img = tf.image.resize(img, [224, 224])  # Resize to 224x224
    img = img / 255.0  # Normalize pixel values
    return img, label

# Create TensorFlow datasets
BATCH_SIZE = 128

train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
train_dataset = train_dataset.map(process_path).shuffle(1000).batch(BATCH_SIZE)

val_dataset = tf.data.Dataset.from_tensor_slices((val_images, val_labels))
val_dataset = val_dataset.map(process_path).batch(BATCH_SIZE)

test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels))
test_dataset = test_dataset.map(process_path).batch(BATCH_SIZE)

# Load ResNet50 without the top classification layer
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the pre-trained layers
for layer in base_model.layers:
    layer.trainable = False

# Add custom classifier on top
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(128, activation='relu')(x)
x = Dense(1, activation='sigmoid')(x)  # Binary classification

# Create the model
model = Model(inputs=base_model.input, outputs=x)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train model
EPOCHS = 100
history = model.fit(train_dataset, validation_data=val_dataset, epochs=EPOCHS)

# Evaluate model
import sklearn.metrics as metrics
from sklearn.metrics import confusion_matrix

# Predict on test data
test_preds = model.predict(test_dataset)
test_preds = (test_preds > 0.5).astype(int).flatten()

# Get confusion matrix values
tn, fp, fn, tp = confusion_matrix(test_labels, test_preds).ravel()

# Calculate IBA
iba = (tp / (tp + fn)) + (tn / (tn + fp)) - 1

# Output results
f1 = f1_score(test_labels, test_preds)
gmean = geometric_mean_score(test_labels, test_preds)

loss, accuracy = model.evaluate(test_dataset)
print(f"Test Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"G-Mean: {gmean:.4f}")
print(f"Informedness (IBA): {iba:.4f}")


Total Benign Images: 649
Total Malignant Images: 1437
Training samples: 1251
Validation samples: 209
Testing samples: 626
Epoch 1/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 245s 7s/step - accuracy: 0.6070 - loss: 0.6994 - val_accuracy: 0.6890 - val_loss: 0.6653
Epoch 2/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 20s 677ms/step - accuracy: 0.7026 - loss: 0.6189 - val_accuracy: 0.6890 - val_loss: 0.6514
Epoch 3/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 22s 975ms/step - accuracy: 0.6961 - loss: 0.6377 - val_accuracy: 0.6890 - val_loss: 0.6128
Epoch 4/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 41s 973ms/step - accuracy: 0.6866 - loss: 0.6136 - val_accuracy: 0.6890 - val_loss: 0.6063
Epoch 5/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 39s 738ms/step - accuracy: 0.7077 - loss: 0.5942 - val_accuracy: 0.6938 - val_loss: 0.6030
Epoch 6/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 20s 687ms/step - accuracy: 0.6929 - loss: 0.6142 - val_accuracy: 0.6890 - val_loss: 0.6050
Epoch 7/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 20s 725ms/step - accuracy: 0.7200 - loss: 0.5891 - val_accura

***
**Magnification Factor: 200X**
***

In [ ]:
import os
import tensorflow as tf
import numpy as np
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from imblearn.metrics import geometric_mean_score

# Define dataset paths
benign_dir = "/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/benign/200X"
malignant_dir = "/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/malignant/200X"

# Function to load image paths
def load_image_paths(dir_path):
    return [os.path.join(dir_path, img) for img in os.listdir(dir_path) if img.endswith('.png')]

benign_images = load_image_paths(benign_dir)
malignant_images = load_image_paths(malignant_dir)

print(f"Total Benign Images: {len(benign_images)}")
print(f"Total Malignant Images: {len(malignant_images)}")

# Create labels (0 = Benign, 1 = Malignant)
benign_labels = [0] * len(benign_images)
malignant_labels = [1] * len(malignant_images)

# Combine images and labels
all_images = np.array(benign_images + malignant_images)
all_labels = np.array(benign_labels + malignant_labels)

# Split into training (70%) and testing (30%)
train_images, test_images, train_labels, test_labels = train_test_split(
    all_images, all_labels, test_size=0.3, stratify=all_labels, random_state=42
)

# Further split training set into 60% training and 10% validation
train_images, val_images, train_labels, val_labels = train_test_split(
    train_images, train_labels, test_size=0.1429, stratify=train_labels, random_state=42
)

print(f"Training samples: {len(train_images)}")
print(f"Validation samples: {len(val_images)}")
print(f"Testing samples: {len(test_images)}")

# Function to preprocess images
def process_path(file_path, label):
    img = tf.io.read_file(file_path)
    img = tf.image.decode_png(img, channels=3)  # Decode PNG images
    img = tf.image.resize(img, [224, 224])  # Resize to 224x224
    img = img / 255.0  # Normalize pixel values
    return img, label

# Create TensorFlow datasets
BATCH_SIZE = 128

train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
train_dataset = train_dataset.map(process_path).shuffle(1000).batch(BATCH_SIZE)

val_dataset = tf.data.Dataset.from_tensor_slices((val_images, val_labels))
val_dataset = val_dataset.map(process_path).batch(BATCH_SIZE)

test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels))
test_dataset = test_dataset.map(process_path).batch(BATCH_SIZE)

# Load ResNet50 without the top classification layer
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the pre-trained layers
for layer in base_model.layers:
    layer.trainable = False

# Add custom classifier on top
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(128, activation='relu')(x)
x = Dense(1, activation='sigmoid')(x)  # Binary classification

# Create the model
model = Model(inputs=base_model.input, outputs=x)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train model
EPOCHS = 100
history = model.fit(train_dataset, validation_data=val_dataset, epochs=EPOCHS)

# Evaluate model
import sklearn.metrics as metrics
from sklearn.metrics import confusion_matrix

# Predict on test data
test_preds = model.predict(test_dataset)
test_preds = (test_preds > 0.5).astype(int).flatten()

# Get confusion matrix values
tn, fp, fn, tp = confusion_matrix(test_labels, test_preds).ravel()

# Calculate IBA
iba = (tp / (tp + fn)) + (tn / (tn + fp)) - 1

# Output results
f1 = f1_score(test_labels, test_preds)
gmean = geometric_mean_score(test_labels, test_preds)

loss, accuracy = model.evaluate(test_dataset)
print(f"Test Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"G-Mean: {gmean:.4f}")
print(f"Informedness (IBA): {iba:.4f}")


Total Benign Images: 623
Total Malignant Images: 1390
Training samples: 1207
Validation samples: 202
Testing samples: 604
Epoch 1/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 211s 5s/step - accuracy: 0.6090 - loss: 0.6762 - val_accuracy: 0.6881 - val_loss: 0.6446
Epoch 2/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 19s 573ms/step - accuracy: 0.6986 - loss: 0.6174 - val_accuracy: 0.6881 - val_loss: 0.6131
Epoch 3/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 25s 910ms/step - accuracy: 0.6909 - loss: 0.6075 - val_accuracy: 0.6881 - val_loss: 0.6270
Epoch 4/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 39s 907ms/step - accuracy: 0.6831 - loss: 0.6320 - val_accuracy: 0.6881 - val_loss: 0.6097
Epoch 5/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 19s 590ms/step - accuracy: 0.6963 - loss: 0.6009 - val_accuracy: 0.6881 - val_loss: 0.6073
Epoch 6/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 21s 720ms/step - accuracy: 0.6879 - loss: 0.6118 - val_accuracy: 0.6881 - val_loss: 0.6045
Epoch 7/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 18s 661ms/step - accuracy: 0.6917 - loss: 0.6052 - val_accura

4/5 ━━━━━━━━━━━━━━━━━━━━ 9s 10s/step 

5/5 ━━━━━━━━━━━━━━━━━━━━ 64s 12s/step
5/5 ━━━━━━━━━━━━━━━━━━━━ 10s 2s/step - accuracy: 0.7125 - loss: 0.5534
Test Accuracy: 0.7053
F1 Score: 0.8231
G-Mean: 0.2524
Informedness (IBA): 0.0570


***
**Magnification Factor: 400X**
***

In [2]:
import os
import tensorflow as tf
import numpy as np
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from imblearn.metrics import geometric_mean_score

# Define dataset paths
benign_dir = "/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/benign/400X"
malignant_dir = "/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/malignant/400X"

# Function to load image paths
def load_image_paths(dir_path):
    return [os.path.join(dir_path, img) for img in os.listdir(dir_path) if img.endswith('.png')]

benign_images = load_image_paths(benign_dir)
malignant_images = load_image_paths(malignant_dir)

print(f"Total Benign Images: {len(benign_images)}")
print(f"Total Malignant Images: {len(malignant_images)}")

# Create labels (0 = Benign, 1 = Malignant)
benign_labels = [0] * len(benign_images)
malignant_labels = [1] * len(malignant_images)

# Combine images and labels
all_images = np.array(benign_images + malignant_images)
all_labels = np.array(benign_labels + malignant_labels)

# Split into training (70%) and testing (30%)
train_images, test_images, train_labels, test_labels = train_test_split(
    all_images, all_labels, test_size=0.3, stratify=all_labels, random_state=42
)

# Further split training set into 60% training and 10% validation
train_images, val_images, train_labels, val_labels = train_test_split(
    train_images, train_labels, test_size=0.1429, stratify=train_labels, random_state=42
)

print(f"Training samples: {len(train_images)}")
print(f"Validation samples: {len(val_images)}")
print(f"Testing samples: {len(test_images)}")

# Function to preprocess images
def process_path(file_path, label):
    img = tf.io.read_file(file_path)
    img = tf.image.decode_png(img, channels=3)  # Decode PNG images
    img = tf.image.resize(img, [224, 224])  # Resize to 224x224
    img = img / 255.0  # Normalize pixel values
    return img, label

# Create TensorFlow datasets
BATCH_SIZE = 128

train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
train_dataset = train_dataset.map(process_path).shuffle(1000).batch(BATCH_SIZE)

val_dataset = tf.data.Dataset.from_tensor_slices((val_images, val_labels))
val_dataset = val_dataset.map(process_path).batch(BATCH_SIZE)

test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels))
test_dataset = test_dataset.map(process_path).batch(BATCH_SIZE)

# Load ResNet50 without the top classification layer
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the pre-trained layers
for layer in base_model.layers:
    layer.trainable = False

# Add custom classifier on top
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(128, activation='relu')(x)
x = Dense(1, activation='sigmoid')(x)  # Binary classification

# Create the model
model = Model(inputs=base_model.input, outputs=x)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train model
EPOCHS = 100
history = model.fit(train_dataset, validation_data=val_dataset, epochs=EPOCHS)

# Evaluate model
import sklearn.metrics as metrics
from sklearn.metrics import confusion_matrix

# Predict on test data
test_preds = model.predict(test_dataset)
test_preds = (test_preds > 0.5).astype(int).flatten()

# Get confusion matrix values
tn, fp, fn, tp = confusion_matrix(test_labels, test_preds).ravel()

# Calculate IBA
iba = (tp / (tp + fn)) + (tn / (tn + fp)) - 1

# Output results
f1 = f1_score(test_labels, test_preds)
gmean = geometric_mean_score(test_labels, test_preds)

loss, accuracy = model.evaluate(test_dataset)
print(f"Test Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"G-Mean: {gmean:.4f}")
print(f"Informedness (IBA): {iba:.4f}")


Total Benign Images: 588
Total Malignant Images: 1232
Training samples: 1091
Validation samples: 183
Testing samples: 546
94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 271s 6s/step - accuracy: 0.5677 - loss: 0.7285 - val_accuracy: 0.6776 - val_loss: 0.6366
Epoch 2/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 14s 516ms/step - accuracy: 0.6768 - loss: 0.6431 - val_accuracy: 0.6776 - val_loss: 0.6256
Epoch 3/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 22s 600ms/step - accuracy: 0.6802 - loss: 0.6252 - val_accuracy: 0.6776 - val_loss: 0.6408
Epoch 4/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 20s 492ms/step - accuracy: 0.6870 - loss: 0.6242 - val_accuracy: 0.6776 - val_loss: 0.6253
Epoch 5/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 21s 642ms/step - accuracy: 0.6826 - loss: 0.6194 - val_accuracy: 0.6776 - val_loss: 0.6192
Epoch 6/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 20s 491ms/step - accuracy: 0.6687 - loss: 0.6251 - val_accuracy: 0.6776 - val_loss: 0.6204
Epoch 7/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 16s 603ms/step - accurac

# Here is a table summarizing the performance metrics for all magnification factors (40X, 100X, 200X, 400X) from notebook:

In [2]:
import pandas as pd

# Create a DataFrame with the provided values
data = {
    'Magnification': ['40X', '100X', '200X', '400X'],
    'Test Accuracy': [0.7095, 0.7125, 0.7053, 0.6905],
    'F1 Score': [0.8214, 0.8189, 0.8231, 0.8124],
    'G-Mean': [0.3597, 0.4346, 0.2524, 0.2486],
    'Informedness (IBA)': [0.1062, 0.1443, 0.0570, 0.0517]
}

df = pd.DataFrame(data)

# Display the table with formatting
styled_df = df.style \
    .format({
        'Test Accuracy': '{:.4f}',
        'F1 Score': '{:.4f}',
        'G-Mean': '{:.4f}',
        'Informedness (IBA)': '{:.4f}'
    }) \
    .set_properties(**{'text-align': 'center'}) \
    .set_table_styles([
        {'selector': 'th', 'props': [('background-color', '#000000'), ('font-weight', 'bold'), ('color', 'white')]}
    ]) \
    .hide(axis='index')

styled_df


Magnification,Test Accuracy,F1 Score,G-Mean,Informedness (IBA)
40X,0.7095,0.8214,0.3597,0.1062
100X,0.7125,0.8189,0.4346,0.1443
200X,0.7053,0.8231,0.2524,0.0570
400X,0.6905,0.8124,0.2486,0.0517
